In [3]:
import os
from groq import Groq


def message(by, to):
    return dict(zip(["role", "content"], [by, to]))


def message_system(to):
    return message("system", to)


def message_user(to):
    return message("user", to)


def message_assistant(to):
    return message("assistant", to)


def chat_messages(messages):
    return (
        messages
        if (
            bool(messages)
            and len(messages) % 2 == 0
            and all(
                messages[i]["role"] == "user" and messages[i + 1]["role"] == "assistant"
                for i in range(0, len(messages), 2)
            )
        )
        else []
    )

In [2]:
import ujson as json


def chat_messages_from_file(file):
    if not os.path.exists(file):
        return []
    with open(file, "r") as f:
        return chat_messages(json.loads(f))


def chat_messages_to_file(messages, file):
    if not os.path.exists(file):
        os.makedirs(os.path.dirname(file), exist_ok=True)
    with open(file, "w") as f:
        return json.dump(chat_messages(messages), f, indent=2, ensure_ascii=False)

client = Groq()

def chat_completion(chat_input, buffer_messages=[]):
    if not chat_messages(buffer_messages):
        buff = chat_messages([message_user(input)])
    else:
        buffer_messages_file = "buffer_messages.json"
        buff = chat_messages_from_file(buffer_messages_file)
    return (
        client.chat.completions.create(
            messages=[message(to=chat_input) + buff],
            model="llama-3.1-8b-instant",
        )
        .choices[0]
        .message.content
    )